## Great Barrier Reef - Training Script

Downloads the specified branch of the repo from the github and can run the training or even a full parameter sweep.

In [1]:
#@title Select your options
mode = "train"    #@param ["train", "evaluate"]
branch_name = "GBR_starfish_detection"    #@param {type:"string"}
wandb_model = "stuttgartteam8/protect_gbr/2pjrq56n"    #@param ["New", "stuttgartteam8/protect_gbr/2pjrq56n"]
full_dataset = True    #@param {type:"boolean"}

In [2]:
!nvidia-smi

Wed Jan  5 14:35:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# download dataset
%cd /content/
if full_dataset:
    !gdown https://drive.google.com/uc?id=1DW2CJja9n8d0Bub-jmJZvnKXAHrpYLkv
else:
    # images 0-478 (part of whole data set)
    !gdown https://drive.google.com/uc?id=1nVROGC31eVwThW7xYUM8zuxlmQVgC9xr


/content
Downloading...
From: https://drive.google.com/uc?id=1DW2CJja9n8d0Bub-jmJZvnKXAHrpYLkv
To: /content/GBR_dataset.zip
100% 15.2G/15.2G [02:45<00:00, 91.9MB/s]


In [4]:
%%capture
# unpack dataset
if full_dataset:
    !unzip -u "/content/GBR_dataset.zip" -d "/content/";
else:
    !unzip -u "/content/GBR_dataset_partial.zip" -d "/content/";

In [5]:
# delete existing repo to ensure the newest is fetched from github 
% cd /
!rm -r /content/dl-lab-21w-team08

# download repo
# %%capture
%cd /content/
git_access_token = "28fdd19659f4a0e5bcaf16dd820e5c68bed91102"

# !git clone https://28fdd19659f4a0e5bcaf16dd820e5c68bed91102@github.tik.uni-stuttgart.de/iss/dl-lab-21w-team08.git
clone_comand = f"git clone --single-branch --branch {branch_name} https://{git_access_token}@github.tik.uni-stuttgart.de/iss/dl-lab-21w-team08.git"
!$clone_comand

/
/content
Cloning into 'dl-lab-21w-team08'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 391 (delta 4), reused 8 (delta 3), pack-reused 378
Receiving objects: 100% (391/391), 6.43 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (246/246), done.


In [6]:
# install dependencies
%%capture
!pip install wandb
!pip install tensorflow_addons
#!pip install gin-config

In [7]:
%%capture out
# run training script
# 37262d20054e8dbf092705158103cd02e31691d6
%cd /content/
!mkdir wandb
%cd /content/dl-lab-21w-team08/GBR_starfish_detection/

!wandb login 37262d20054e8dbf092705158103cd02e31691d6

In [8]:
wandb_model

'stuttgartteam8/protect_gbr/2pjrq56n'

In [ ]:
# training

if mode == "train":
    # pass dataset path and the wandb model which is used as starting point for training
    if full_dataset:
        !python main.py -p="/content/GBR_dataset/" --wandb_model={wandb_model} --dataset_slice_end=23500
    else:
        # use only 200 images for training
        !python main.py -p="/content/GBR_dataset_partial/" --wandb_model={wandb_model} --dataset_slice_end=300

In [11]:
# model evaluation

%cd /content/dl-lab-21w-team08/GBR_starfish_detection/
import wandb
import tensorflow as tf
from input import load, annotate_image

wandb.init(project="protect_gbr", entity="stuttgartteam8", mode="disabled") 
if full_dataset:
    arg_update = {"data_dir": "/content/GBR_dataset/"}
else:
    arg_update = {"data_dir": "/content/GBR_dataset_partial/"}
wandb.config.update(arg_update, allow_val_change=True)
config = wandb.config

ds_train, ds_val, ds_test = load(config)

if mode == "evaluate":
    # download the model from wandb
    print("Downloading model")
    api = wandb.Api()
    run = api.run(wandb_model)
    run.file("model.h5").download(replace=True)

    print("Loading model")
    model = tf.keras.models.load_model('model.h5', compile=False) 
    print(model.summary())

else:
    # use the model from this training session and evaluate it

    file = "/content/dl-lab-21w-team08/GBR_starfish_detection/wandb/latest-run/files/model.h5"
    model = tf.keras.models.load_model(file, compile=False) # todo: the path for the model needs to be adapted!

/content/dl-lab-21w-team08/GBR_starfish_detection
   video_id  sequence  video_frame  sequence_frame image_id annotations
0         0     40258            0               0      0-0          []
1         0     40258            1               1      0-1          []
2         0     40258            2               2      0-2          []
3         0     40258            3               3      0-3          []
4         0     40258            4               4      0-4          []


In [ ]:

import tensorflow.keras as keras
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import cv2

for x,y in ds_train:
    y_pred = model.predict(x)
    img = annotate_image(config, x[0], y_pred[0], y[0])
    img = img.numpy()*255
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2_imshow(img)
    break